<a href="https://colab.research.google.com/github/PARTHIBAN-007/Hands-ON-LLMs/blob/main/6_Prmopt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color:#ffecd2; color:#ad5389; text-align:center; padding:15px; border-radius:25px;">Prompt Engineering</div>

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline


model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-3-mini-4k-instruct",
    device_map="cuda",
    trust_remote_code =True,
    torch_dtype ="auto"
)


tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct")


pipe = pipeline(
    "text-generation",
    model =model ,
    tokenizer = tokenizer ,
    max_new_tokens =50,
    do_sample =False,
    return_full_text =False

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
messages = [
    {"role":"user","content":"Create a funny joke about thr life of Data Scientists and Ml Enginneers"}
]


output = pipe(messages)

print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why did the data scientist get kicked out of the bar? Because every time he tried to explain his model, he kept saying, "It's not rocket science, but it's definitely not a simple linear regression either!"


In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages,tokenize =False)

print(prompt)

<|user|>
Create a funny joke about thr life of Data Scientists and Ml Enginneers<|end|>
<|endoftext|>


In [ ]:

output = pipe(messages,do_sample =True , temperature =1)

print(output[0]["generated_text"])

 Why do every data scientist and machine learning engineer need a therapist? 

Because they're used to dealing with hard data. Sometimes, the complexity of algorithms and the stress of constant validation and deployment can be quite overwh


In [ ]:

output = pipe(messages,do_sample =True ,top_p =1)

print(output[0]["generated_text"])

 Sure! Here's a funny, light-hearted joke about the lives of Data Scientists and Machine Learning Engineers:

Why do Data Scientists and Machine Learning Engineers avoid small talk at parties?

Because


**Intro To Prompt Engineering**

In [ ]:
text = """In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud’s recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let’s try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard’s NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let’s begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other – there’s nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder’s inputs first flow through a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We’ll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we’ve seen the major components of the model, let’s start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 – In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below. The size of this list is hyperparameter we can set – basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we’ll switch up the example to a shorter sentence and we’ll look at what happens in each sub-layer of the encoder.
Now We’re Encoding!
As we’ve mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a ‘self-attention’ layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""


persona = "You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.\n"
instruction = "Summarize the key findings of the paper provided.\n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.\n"
data_format = "Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.\n"
audience = "The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.\n"
tone = "The tone should be professional and clear.\n"
text = "MY TEXT TO SUMMARIZE"
data = f"Text to summarize: {text}"



query = persona + instruction + context + data_format + audience + tone + data

In [ ]:
messages = [
    {"role": "user", "content": query}
]
print(tokenizer.apply_chat_template(messages, tokenize=False))

In [ ]:
outputs = pipe(messages)
print(outputs[0]["generated_text"])

## One Shot Prompting

In [ ]:
one_shot_prompt = [
    {
        "role": "user",
        "content": "An LLM, or Large Language Model, is a type of AI designed to generate text and understand natural language. An example of using an LLM in a sentence is:"
    },
    {
        "role": "assistant",
        "content": "I used an LLM to help me draft an email quickly and accurately."
    },
    {
        "role": "user",
        "content": "A use case of an LLM in education is:"
    }
]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
An LLM, or Large Language Model, is a type of AI designed to generate text and understand natural language. An example of using an LLM in a sentence is:<|end|>
<|assistant|>
I used an LLM to help me draft an email quickly and accurately.<|end|>
<|user|>
A use case of an LLM in education is:<|end|>
<|endoftext|>


In [ ]:
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 In an educational setting, an LLM can be utilized to create personalized learning experiences for students. For instance, an LLM can analyze a student's learning style and performance data to generate customized study materials and quizzes that c


### Chain Prompting

In [ ]:
product_prompt = [
    {"role": "user", "content": "Create a name and slogan for a chatbot that leverages LLMs."}
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

 Name: ChatSage
Slogan: "Unleashing the power of AI to enhance your conversations."


In [ ]:
sales_prompt = [
    {"role": "user", "content": f"Generate a very short sales pitch for the following product: '{product_description}'"}
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

 Introducing ChatSage, the revolutionary AI-powered tool designed to elevate your conversations to new heights. With our cutting-edge technology, we unleash the power of AI to enhance your interactions


## Chain of Thought Prompting

<img src="https://promptengineering.org/content/images/2023/04/image-11.png">

In [ ]:
standard_prompt = [
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
     {"role": "assistant", "content": "11"},
     {"role": "user", "content": "The cafeteria had 25 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
 ]

outputs = pipe(standard_prompt)
print(outputs[0]["generated_text"])

 The cafeteria started with 25 apples. They used 20 apples to make lunch, so they had 25 - 20 = 5 apples left. After buying 6 more apples


In [ ]:
cot_prompt = [
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"role": "assistant", "content": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
]

outputs = pipe(cot_prompt)
print(outputs[0]["generated_text"])

 The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. After buying 6 more apples,


## Zero Shot Prompting

In [7]:
zeroshot_cot_prompt = [
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step-by-step."}
]

outputs = pipe(zeroshot_cot_prompt,return_full_text =True)
print(outputs[0]["generated_text"])

[{'role': 'user', 'content': "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step-by-step."}, {'role': 'assistant', 'content': ' Step 1: Start with the initial number of apples in the cafeteria, which is 23.\n\nStep 2: Subtract the number of apples used to make lunch, which is 20'}]


## Tree of Thought

In [1]:
zeroshot_tot_prompt = [
    {"role":"user",
     "content":'''Imagine three different experts are answering this question.
      All experts will write down 1 step of their thinking, then share it with
     the group. Then all experts will go on to the next step, etc.
     If any expert realises they're wrong at any point then they leave.
      The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more,
       how many apples do they have?' Make sure to discuss the results.'''}

]

In [6]:
output = pipe(zeroshot_tot_prompt)
print(output[0]["generated_text"])

 Expert 1:
Step 1: Start with the initial number of apples, which is 23.

Expert 2:
Step 1: Subtract the number of apples used for lunch,


In [ ]:
zeroshot_prompt = [
    {"role": "user", "content": "Create a character profile for an RPG game in JSON format."}
]

outputs = pipe(zeroshot_prompt)
print(outputs[0]["generated_text"])

 ```json
{
  "name": "Eldrin the Wise",
  "race": "Elf",
  "class": "Wizard",
  "level": 10,
  "alignment":


In [ ]:
one_shot_template = """Create a short character profile for an RPG game. Make sure to only use this format:

{
  "description": "A SHORT DESCRIPTION",
  "name": "THE CHARACTER'S NAME",
  "armor": "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}
"""
one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]

outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 {
  "description": "A cunning rogue with a mysterious past, skilled in stealth and deception.",
  "name": "Shadowcloak",
  "armor": "Leather Hood",


In [13]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.6-cp310-cp310-linux_x86_64.whl size=4022120 sha256=112898cc0e70dc8e886a9663d16d07276b638399e5b92020518cd7b97fa4e9de
  Stored in directory: /root/.cache/pip/wheels/06/61/7b/d5ebacdccd230f42c847b6eec707a26c8455f49277393b295c
Successfully built llama-cpp-python


In [15]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [16]:
from llama_cpp.llama import Llama

# Load Phi-3
llm = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=False
)

Phi-3-mini-4k-instruct-fp16.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s]

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [18]:
output = llm.create_chat_completion(
    messages=[
        {"role": "user", "content": "Create a warrior for an RPG in JSON format."},
    ],
    response_format={"type": "json_object"},
    temperature=0,
)['choices'][0]['message']["content"]

In [19]:
import json
json_output = json.dumps(json.loads(output),indent =1)
print(json_output)

{
 "warrior": {
  "name": "Eldric Stormbringer",
  "class": "Warrior",
  "level": 5,
  "attributes": {
   "strength": 18,
   "dexterity": 10,
   "constitution": 16,
   "intelligence": 8,
   "wisdom": 10,
   "charisma": 12
  },
  "skills": [
   {
    "name": "Martial Arts",
    "proficiency": 20,
    "description": "Expert in hand-to-hand combat and weapon handling."
   },
   {
    "name": "Shield Block",
    "proficiency": 18,
    "description": "Highly skilled at deflecting attacks with a shield."
   },
   {
    "name": "Heavy Armor",
    "proficiency": 16,
    "description": "Expertly equipped with heavy armor for protection."
   },
   {
    "name": "Survival",
    "proficiency": 14,
    "description": "Adept at finding food, water, and shelter in the wilderness."
   }
  ],
  "equipment": [
   {
    "name": "Iron Sword",
    "type": "Weapon",
    "damage": 12,
    "durability": 100
   },
   {
    "name": "Chainmail Armor",
    "type": "Armor",
    "defense": 18,
    "durability": 100